In [ ]:
from matplotlib import pyplot
%matplotlib inline
import numpy as np
from scipy.interpolate import interp1d
import ytree

from ytree.data_structures.tree_container import TreeContainer
from yt.extensions.p2p.model_profiles import get_star_data
from yt.extensions.p2p.tree_analysis_operations import get_progenitor_line

In [ ]:
pyplot.rcParams['figure.figsize'] = (15, 9)
pyplot.rcParams['font.size'] = 20

In [ ]:
star_data = get_star_data("star_hosts.yaml")

In [ ]:
star_data.keys()

In [ ]:
my_star = star_data[334267099]

In [ ]:
a = ytree.load(my_star["arbor"])

In [ ]:
my_root = a[my_star["_arbor_index"]]
my_tree = my_root.get_node("forest", my_star["tree_id"])
ct = my_star["creation_time"]

In [ ]:
prog = TreeContainer(a, get_progenitor_line(my_tree))

In [ ]:
good = (prog["time"] <= ct) | ((prog["time"] > ct) & (prog["mass"] > 1e4))

In [ ]:
### Using creation time
i1 = np.digitize(ct, prog["time"][good])
i2 = np.where(good)[0][-1]
my_x = np.array([prog["time"][good][i1], prog["time"][i2]])
my_y = np.log10([prog["mass"][good][i1], prog["mass"][i2]])
ikwargs = {"kind": "linear", "fill_value": "extrapolate"}
f1 = interp1d(my_x, my_y, **ikwargs)

In [ ]:
i1 = np.where(good)[0][-20]
i2 = np.where(good)[0][-10]
my_x = np.array([prog["time"][i1], prog["time"][i2]])
my_y = np.log10([prog["mass"][i1], prog["mass"][i2]])
ikwargs = {"kind": "linear", "fill_value": "extrapolate"}
f1 = interp1d(my_x, my_y, **ikwargs)

In [ ]:
my_x, my_y

In [ ]:
slope = (my_y[1] - my_y[0]) / (my_x[1] - my_x[0])
my_lmf = 7
my_tf = (my_lmf - my_y[0]) / slope + my_x[0]

In [ ]:
tf = prog["time"][good][-1]
tff = a.quan(my_tf, "Myr")
mf = prog["mass"][good][-1]
mff = a.quan(np.power(10, f1(tff)), mf.units)

In [ ]:
tff

In [ ]:
pyplot.semilogy(prog["time"], prog["mass"])
pyplot.semilogy(prog["time"][good], prog["mass"][good])
pyplot.semilogy([tf, tff], [mf, mff])
pyplot.axvline(my_star["creation_time"])
pyplot.ylabel("M [M$_{\odot}$]")
pyplot.xlabel("t [Myr]")
pyplot.show()